In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from models import BasicRLModel, SR_RLModel
from utils.data_preparation_utils import process_data
from utils.utils import create_value_graph, gen_nbackstructure_trials, gen_structured_RL_trials

%matplotlib inline

Helper Functions

In [2]:
def rep_simulate(model, reps=50, data=None, query_str=None):
    acc = []
    for _ in range(reps):
        out=model.simulate(data=data)
        if query_str:
            out = out.query(query_str)
        acc.append(out.correct.mean())
    return np.mean(acc)

def eval_models(models, data):
    for name, model in models:
        blank_acc = rep_simulate(model, data=data, query_str="display_reward==0 and stim_set != (1,6,11)")
        acc = rep_simulate(model, data=data)
        print('*')*79
        print(name)
        print({k:round(v, 3) for k,v in model.get_params().items()})
        print('Accuracy: %s, Blank Acc: %s' % (acc, blank_acc))

# Data

## From a subject

In [3]:
subj_data = process_data('651')
subj_RL = subj_data['RL']
subj_structure = subj_data['structure']
subj_meta = subj_data['meta']
subj_descriptive_stats = subj_data['descriptive_stats']

## Simulated

In [4]:
# generate an RL task
graph = {0: [1,2,3,14],
         1: [0,2,3,4],
         2: [0,1,3,4],
         3: [0,1,2,4],
         4: [1,2,3,5],
         5: [4,6,7,8],
         6: [5,7,8,9],
         7: [5,6,8,9],
         8: [5,6,7,9],
         9: [6,7,8,10],
         10: [9,11,12,13],
         11: [10,12,13,14],
         12: [10,11,13,14],
         13: [10,11,12,14],
         14: [11,12,13,0]}
stims = ['images/%s.png' % str(i+1) for i in graph.keys()]
np.random.seed(2222)
seeds = {1:1, 7:0}
values = create_value_graph(graph, seeds, weight=.97, steps = 3000,
                            scaling=.7, offset=.15)

np.random.seed()

RLdata = pd.DataFrame(gen_structured_RL_trials(stims, values, repeats=20, reward_blackout=10))
structuredata = pd.DataFrame(gen_nbackstructure_trials(graph, stims, 1400))

# shuffle values
vals = values.values()
np.random.shuffle(vals)
for key in values.keys():
    values[key] = vals.pop()
shuffled_RLdata = pd.DataFrame(gen_structured_RL_trials(stims, values, repeats=20, reward_blackout=10))

#random values
for key in values.keys():
    values[key] = np.random.randn()
random_RLdata = pd.DataFrame(gen_structured_RL_trials(stims, values, repeats=20, reward_blackout=10))

# Fit Models

First fit models on the actual data, and evaluate performance on the data, as well as the random and shuffled data

In [5]:
subj_m = BasicRLModel(subj_RL)
subj_m.optimize()
m = BasicRLModel(RLdata)
m.optimize_acc()
SRm = SR_RLModel(RLdata, structuredata)
SRm.optimize_acc()

In [7]:
print('Subject Accuracy: %s' % subj_RL.correct.mean())
print('\n' + '*'*79 + '\n')
print('Model Performance')
models = [['optimized SR', SRm],['subject', subj_m], ['optimized RL', m]]
eval_models(models, data=RLdata)

print('\n' + '*'*79 + '\n')
print('Model Performance on Shuffled values')
eval_models(models, data=shuffled_RLdata)

print('\n' + '*'*79 + '\n')
print('Model Performance on Random values')
eval_models(models, data=random_RLdata)

Subject Accuracy: 0.90234375

*******************************************************************************

Model Performance
*******************************************************************************
optimized SR
{'beta': 38.042, 'lr': 0.071, 'eps': 0.087, 'gamma': 0.956, 'SR_lr': 0.024}
Accuracy: 0.926125, Blank Acc: 0.854666666667
*******************************************************************************
subject
{'beta': 10.35, 'lr': 0.136, 'eps': 0.0}
Accuracy: 0.814083333333, Blank Acc: 0.488666666667
*******************************************************************************
optimized RL
{'beta': 13.643, 'lr': 0.075, 'eps': 0.011}
Accuracy: 0.820416666667, Blank Acc: 0.497333333333

*******************************************************************************

Model Performance on Shuffled values
*******************************************************************************
optimized SR
{'beta': 38.042, 'lr': 0.071, 'eps': 0.087, 'gamma': 0.956, 'SR_lr': 0.024

Evaluate the SRm Model params after being separately fit on the different data types

In [ ]:
datasets = [('original', RLdata), ('shuffled', shuffled_RLdata), ('random', random_RLdata)]
for name, data in datasets:
    SRm = SR_RLModel(data, structuredata)
    SRm.optimize_acc()
    print('\n' + '*'*79 + '\n')
    print('Model Performance on %s data' % name)
    eval_models(models, data=data)

The actual transition probability matrix can be calculated from the one-step transition probabilities, given a discount rate gamma. Below I do a grid search across gamma and learning rate values and compare the learning transition matrix (using SR_TD) to the asymptotic matrix

In [ ]:
from models import SR_from_transition
from utils.graph_utils import graph_to_matrix
from utils.utils import get_lower

gamma_list = np.linspace(0,1,11)
LR_list = np.linspace(0,.25,6)

param_grid = np.zeros((len(gamma_list), len(LR_list)))

for i, gamma in enumerate(gamma_list):
    # get asymptotic M
    graph = meta['structure']['graph']
    onestep = graph_to_matrix(graph)/5.0
    TrueM = SR_from_transition(onestep, SRm.gamma)
    for j, lr in enumerate(LR_list):
        SRm.update_params({'SR_lr': lr, 'gamma': gamma})
        M = SRm.SR_TD()
        corr = np.corrcoef(get_lower(M),get_lower(TrueM))[0,1]
        param_grid[i, j] = corr

In [ ]:
sns.heatmap(param_grid, xticklabels=LR_list, yticklabels=gamma_list)